Below is scraping the Treasure rolling tables:

In [1]:
# import urllib.request
# import pandas as pd
# import re

# fp = urllib.request.urlopen("https://dungeonmastertools.github.io/index.html")

# mybytes = fp.read()

# html = mybytes.decode("utf8")

# Total = pd.DataFrame()

# tablestr ='class=\"table-title\">\n\t\t\t\t\t\t\t<h3>'

# tableNameStarts =[m.start() for m in re.finditer(tablestr,html)]    # find all the occurances of HTML that looks like its making a new table
# tablestrLen = len(tablestr)
# newTableNameStarts = []

# for each in tableNameStarts:
#     newTableNameStarts.append(each +tablestrLen)                    # when does the table title declaration end?

# tablenames = []
# for each in range(len(newTableNameStarts)):
#     tableName = html[int(newTableNameStarts[each]):]        # start a string at the end of the table title declaration!!
#     namelength = tableName.find('<')                        # find what loc the html code picks back up (str cut off point)
#     tableName = tableName[:namelength]                      # cut off the new string at that point
#     tablenames.append(tableName)                            # create a list of table names that are in the url

# print(tablenames)

# for i in range(50):                                         #scrapes up to 50 tables
    
#     try:         
#         df = pd.read_html(html)[i]
#         df['TableName'] = tablenames[i]                                 # add the table name as a column
    
#         if i == 0:
#             Total = pd.concat([df, Total], ignore_index=True, axis=0)

        
#         elif i > 0 and Total.columns.to_list() == df.columns.to_list():
#             Total = pd.concat([df, Total], ignore_index=True, axis=0)
    
#     except IndexError:
#         df = pd.read_html(html)[0]

# fp.close()

# Total.sample(10)

In [2]:
#### THIS SEEMS TO SCRAPE A LOT OF DATA, ACTIONS HAVE DISAPPEARED...
#### HOW CAN I PULL AN INDIVIDUAL MONSTER EG, A GOBIN?

def getMonsterdf():
    import requests
    import json
    import pandas as pd
    # global monsterdf

    responseAPI=requests.get("https://api.open5e.com/monsters/?page=1")
    data = responseAPI.text
    jsondata = json.loads(data)
    results=jsondata['results']
    monsterdf= pd.DataFrame(results)
    i=2
    for each in range(100):
        try: 
            responseAPI=requests.get(f"https://api.open5e.com/monsters/?page={i}")
            data = responseAPI.text
            jsondata = json.loads(data)
            results=jsondata['results']
            df = pd.DataFrame(results)
            monsterdf= pd.concat([df,monsterdf],ignore_index=True, axis=0)
        except:
            break
        i+=1
    
    def convert_to_float(frac_str):
        try:
            return float(frac_str)
        except ValueError:
            num, denom = frac_str.split('/')
            try:
                leading, num = num.split(' ')
                whole = float(leading)
            except ValueError:
                whole = 0
            frac = float(num) / float(denom)
            return whole - frac if whole < 0 else whole + frac
    i=0
    for each in monsterdf['challenge_rating']:
        monsterdf['challenge_rating'][i] =  convert_to_float(each)
        i+=1
    
    monsterdf=monsterdf[monsterdf['challenge_rating']!=0]
    return monsterdf
    
    


In [3]:
def getOurMonsters(encounter_challenge_rating):
    import pandas as pd
    import random as rn

    monsterdf = getMonsterdf()
    encounterMonsters = pd.DataFrame()
    remaining_challenge_rating = encounter_challenge_rating

    while encounter_challenge_rating >= remaining_challenge_rating >0 :
        elligiblemonstersdf = monsterdf[monsterdf['challenge_rating']<=remaining_challenge_rating] # filter DF to only monsters w low enough challenge ratings for there to be 5 monsters?

        index = rn.randint(0,len(elligiblemonstersdf.index)-1)
        ourmonster = elligiblemonstersdf.iloc[[index]]

        encounterMonsters = pd.concat([encounterMonsters,ourmonster])
        remaining_challenge_rating= remaining_challenge_rating-sum(ourmonster.challenge_rating)
    print(remaining_challenge_rating)

    return encounterMonsters
        

In [125]:
encounterMonsters = getOurMonsters(1)

#### output pdf with a page for each row
#### give stats and monster info
    #### give lists of speeds, skills, or actions 


0.0


C:\Users\622379\AppData\Local\Temp\1\ipykernel_64300\2553832289.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  monsterdf['challenge_rating'][i] =  convert_to_float(each)


In [126]:
encounterMonsters = pd.concat([encounterMonsters]*3, ignore_index=True)

In [127]:
import pandas as pd
from fpdf import FPDF
import datetime as dt
import numpy as np
Today = str(dt.date.today())
filepath = f'C://Users//622379//Documents//Python//Homebrew//Monster Generator//{Today}.csv'

pdf = FPDF('P', 'mm', 'Letter')

#add page
pdf.add_page()

#specify font
# fonts ('times', 'courier', helvetica', 'symbol', 'zpfdingbats')
# 'B' (bold), 'U' (underline), 'I' (italics)

pdf.set_font('helvetica')
#cell(width, height, ln - go to next line)
#pdf.cell(40,10,'Hello World!', ln=True)


#for i in range(1,41):
#    pdf.cell(0,10, f'This is line {i}', ln=True)

#pdf.set_auto_page_break(auto=True, margin= 15)

# class PDF(FPDF):
#         def header(self):
            #self.image(,10,8,25)
pdf.set_font('helvetica', 'B', 18)
pdf.cell(0,0, f"{Today}'s Characters:", border=False, ln=1, align='L')
pdf.ln(10)
pdf.set_font('helvetica','', 12)

# pdf.cell(0,0, f"{Today}'s Characters:", border=False, ln=1, align='L')
# encounterMonsters=encounterMonsters.reset_index()

for i in encounterMonsters.index:

    Name = encounterMonsters['name'].iloc[i] # print(Name.to_string(index=False))
    cr = encounterMonsters['challenge_rating'].iloc[i]
    Size = encounterMonsters['size'].iloc[i]
    Type = encounterMonsters['type'].iloc[i]
    Subtype = encounterMonsters['subtype'].iloc[i]
    alignment = encounterMonsters['alignment'].iloc[i]
    ac = encounterMonsters['armor_class'].iloc[i]
    hp = encounterMonsters['hit_points'].iloc[i]

    Strength = encounterMonsters['strength'].iloc[i]
    Dexterity = encounterMonsters['dexterity'].iloc[i]
    Constitution = encounterMonsters['constitution'].iloc[i]
    Intelligence = encounterMonsters['intelligence'].iloc[i]
    Wisdom = encounterMonsters['wisdom'].iloc[i]
    Charisma = encounterMonsters['charisma'].iloc[i]
    skills = encounterMonsters['skills'].iloc[i]

    resistances =encounterMonsters['damage_resistances'].iloc[i]
    # if resistances == "":
    #     resistances = "N/A"
    immunities = encounterMonsters['damage_immunities'].iloc[i]
    # if immunities == "":
    #     immunities = "N/A"
    condimmunities = encounterMonsters['condition_immunities'].iloc[i]
    # if condimmunities == "":
    #     condimmunities = "N/A"
    # skilltable = pd.DataFrame(skills)


    pdf.cell(0,0,f"Monster: {Name} ({cr})")
    pdf.cell(0,0,f" AC: {ac}   HP: {hp}", align='R')
    pdf.ln(5)
    pdf.cell(0,0,f"     Type: {Type} {Subtype}")
    pdf.ln(5)
    pdf.cell(0,0,f"     ({alignment})")
    pdf.ln(5)
    pdf.cell(0,0,f"     Size: {Size}")
    pdf.ln(5)
    pdf.ln(5)
    pdf.cell(0,0,f"Strength: {Strength}")
    pdf.ln(5)
    pdf.cell(0,0,f"Dexterity: {Dexterity}")
    pdf.ln(5)
    pdf.cell(0,0,f"Constitution: {Constitution}")
    pdf.ln(5)
    pdf.cell(0,0,f"Intelligence: {Intelligence}")
    pdf.ln(5)
    pdf.cell(0,0,f"Wisdom: {Wisdom}")
    pdf.ln(5)
    pdf.cell(0,0,f"Charisma: {Charisma}")
    pdf.ln(10)
    pdf.cell(0,0,f"Skills: {skills}")
    pdf.ln(5)
    pdf.cell(0,0,f"Resistances: {resistances}")
    pdf.ln(5)
    pdf.cell(0,0,f"Immunities: {immunities}")
    pdf.ln(5)
    pdf.cell(0,0,f"Condition Immunities: {condimmunities}")
    
    saves=[]
    for column in encounterMonsters:
        if 'save' in column:
            # print()
            try:
                int(encounterMonsters[column].iloc[i])
                saves.append(f"{column}: {encounterMonsters[column].iloc[i]}")
            except:
                ""

    # saves = "".join(saves)
    pdf.ln(5)
    pdf.cell(0,0,f"Saves: {saves}")
    pdf.ln(10)

    
    pdf.add_page()



pdf.output(f"C://Users//622379//Documents//Python//Homebrew//Monster Generator//{Today}'s Enemies.pdf")

In [105]:
encounterMonsters.to_clipboard()